In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from fastprogress import progress_bar
import numpy as np
from matplotlib import pyplot as plt
import os
import pandas as pd
from PIL import Image
import re
import shutil
import urllib.request

In [3]:
images_path = '../data/user_images/'
labels_path = '../data/user_labels.csv'
target_size = 640

In [7]:
df = pd.DataFrame()
i, step = 0, 500
while True:
    batch = pd.read_json(
        f""
    ).dropna()
    
    i += step
    print(f"Loaded {i} images")
    
    if batch.empty:
        break
    
    if df.empty:
        df = batch
    else: 
        df = df.append(batch)

Loaded 500 images
Loaded 1000 images
Loaded 1500 images
Loaded 2000 images
Loaded 2500 images
Loaded 3000 images
Loaded 3500 images
Loaded 4000 images
Loaded 4500 images
Loaded 5000 images
Loaded 5500 images
Loaded 6000 images
Loaded 6500 images
Loaded 7000 images
Loaded 7500 images
Loaded 8000 images
Loaded 8500 images
Loaded 9000 images
Loaded 9500 images
Loaded 10000 images
Loaded 10500 images
Loaded 11000 images
Loaded 11500 images
Loaded 12000 images
Loaded 12500 images
Loaded 13000 images
Loaded 13500 images
Loaded 14000 images
Loaded 14500 images
Loaded 15000 images
Loaded 15500 images
Loaded 16000 images
Loaded 16500 images
Loaded 17000 images
Loaded 17500 images
Loaded 18000 images
Loaded 18500 images
Loaded 19000 images
Loaded 19500 images
Loaded 20000 images
Loaded 20500 images
Loaded 21000 images
Loaded 21500 images
Loaded 22000 images
Loaded 22500 images
Loaded 23000 images
Loaded 23500 images
Loaded 24000 images


In [8]:
def get_floats_from_stringy_tup(string):
    p = re.compile(r'\d+\.\d+|\d+')   
    return p.findall(string)

def split_array(array):
    return float(array[0]), target_size-float(array[1])

for col in list(df.columns[1:12]):
    df[col] = df[col].apply(get_floats_from_stringy_tup)
    df[col+'_x'], df[col+'_y'] = zip(*df[col].map(split_array))
    df = df.drop(col, axis=1)

uploadImageDf = pd.DataFrame(df['uploadedImage'].values.tolist())
df['hash'] = uploadImageDf['hash'].values

In [17]:
!rm -rf ../data/user_images/*

In [18]:
for i in progress_bar(range(df.shape[0])):
    try:
        urllib.request.urlretrieve(
            df.iloc[i]['uploadedImage']['url'], 
            f"{images_path}/{df.iloc[i]['hash']}.png"
        )
    except Exception as e:
        with open('failed_urls.txt', 'a') as f:
            f.write(f"{df.iloc[i]['hash']}\n")

In [19]:
df = df.drop('uploadedImage', axis=1)

def add_png(hashes):
    return hashes + '.png'

df['hash'] = df['hash'].apply(add_png)

df = df.rename(columns={"hash":"filename"})
df = df[df.columns[-23:]]

In [20]:
df

,leftEyePoint_x,leftEyePoint_y,rightEyePoint_x,rightEyePoint_y,centreMouthPoint_x,centreMouthPoint_y,leftMouthPoint_x,leftMouthPoint_y,rightMouthPoint_x,rightMouthPoint_y,...,chinPoint_y,leftHeadPoint_x,leftHeadPoint_y,rightHeadPoint_x,rightHeadPoint_y,leftEarPoint_x,leftEarPoint_y,rightEarPoint_x,rightEarPoint_y,filename
0,176.231884,319.227053,417.906594,325.668261,352.979058,488.502415,294.235097,517.874396,401.417054,502.930749,...,620.933962,124.186788,513.236715,471.497585,484.380024,106.151353,275.684372,432.850242,249.404179,6310b3dbaa3549ea90badf31c7bee933.png
1,169.017706,206.119155,372.045073,254.557150,224.669879,480.772947,185.507246,487.471812,262.286619,499.323671,...,533.848623,96.360701,304.541063,409.661836,355.555556,154.589372,119.549106,438.518503,192.721401,d65b1920c7ea4c61bd7ab8709b7e6d1c.png
2,212.560386,194.009662,317.681159,231.111111,222.608696,300.676329,193.752005,310.209324,247.342995,324.122368,...,341.127198,172.367150,211.014493,355.555556,285.990338,224.669879,48.437995,432.334928,181.384855,943a14d469664950a9d4168eee6e2291.png
3,267.954904,287.536232,408.631232,272.592585,364.315604,481.803527,328.760049,484.380024,395.748792,479.227053,...,539.001595,194.267295,340.096618,468.921087,343.703696,164.380024,39.162633,431.819638,14.428334,e7fbe08bb2fe48149ffd829a4a4ddd00.png
4,198.904976,201.996764,290.628019,197.359083,250.434783,312.785821,219.516908,320.515290,275.169082,313.816425,...,357.616739,148.405797,288.566812,349.887271,265.893720,110.789034,135.523334,385.442826,90.177126,b9db262dc66f4ba0b529559f64e0d7d6.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,257.500000,175.176152,318.333333,183.848238,271.666667,232.411924,252.500000,223.306233,298.750000,231.978320,...,248.455285,211.666667,155.663957,379.583333,191.219512,215.000000,69.810298,387.916667,102.330623,dbc4fb0f87ea4ff98a8c32ee2a022591.png
116,228.792271,136.038647,346.280193,143.768116,246.666667,245.420054,298.333333,273.604336,218.333333,278.373984,...,300.921409,177.500000,120.542005,504.583333,202.493225,225.833333,0.867209,470.833333,0.867209,632e2e78d32d447fa584a53bca33fdb1.png
117,256.103044,285.990338,339.581305,308.663430,271.666667,368.563686,246.250000,367.262873,309.166667,386.341463,...,399.349593,172.916667,265.799458,439.583333,342.113821,230.000000,109.701897,462.500000,170.840108,708029e4c71946fb8004602423884175.png
118,251.666667,329.105691,363.333333,300.487805,320.416667,350.352304,270.416667,395.013550,370.833333,365.528455,...,380.160000,216.666667,415.392954,427.083333,339.945799,140.833333,291.382114,404.166667,189.051491,77f8e7e784a84965a81f5257299f96e8.png


In [21]:
df.to_csv(labels_path, index=False)

In [22]:
!rm -rf '../data/resized_and_user_images/*'

for src in ['user', 'resized']:
    src_files = os.listdir(f'../data/{src}_images/')
    for file_name in progress_bar(src_files):
        full_file_name = os.path.join(f'../data/{src}_images/', file_name)
        if os.path.isfile(full_file_name):
            im = Image.open(full_file_name)
            im.thumbnail((320,320), Image.ANTIALIAS)
            im.save(f'../data/resized_and_user_images/{file_name}', "JPEG")
#             shutil.copy(full_file_name, '../data/resized_and_user_images')

In [23]:
user_df = pd.read_csv(f'../data/user_labels.csv')
resized_df = pd.read_csv(f'../data/resized_labels.csv')

In [24]:
user_df

,leftEyePoint_x,leftEyePoint_y,rightEyePoint_x,rightEyePoint_y,centreMouthPoint_x,centreMouthPoint_y,leftMouthPoint_x,leftMouthPoint_y,rightMouthPoint_x,rightMouthPoint_y,...,chinPoint_y,leftHeadPoint_x,leftHeadPoint_y,rightHeadPoint_x,rightHeadPoint_y,leftEarPoint_x,leftEarPoint_y,rightEarPoint_x,rightEarPoint_y,filename
0,176.231884,319.227053,417.906594,325.668261,352.979058,488.502415,294.235097,517.874396,401.417054,502.930749,...,620.933962,124.186788,513.236715,471.497585,484.380024,106.151353,275.684372,432.850242,249.404179,6310b3dbaa3549ea90badf31c7bee933.png
1,169.017706,206.119155,372.045073,254.557150,224.669879,480.772947,185.507246,487.471812,262.286619,499.323671,...,533.848623,96.360701,304.541063,409.661836,355.555556,154.589372,119.549106,438.518503,192.721401,d65b1920c7ea4c61bd7ab8709b7e6d1c.png
2,212.560386,194.009662,317.681159,231.111111,222.608696,300.676329,193.752005,310.209324,247.342995,324.122368,...,341.127198,172.367150,211.014493,355.555556,285.990338,224.669879,48.437995,432.334928,181.384855,943a14d469664950a9d4168eee6e2291.png
3,267.954904,287.536232,408.631232,272.592585,364.315604,481.803527,328.760049,484.380024,395.748792,479.227053,...,539.001595,194.267295,340.096618,468.921087,343.703696,164.380024,39.162633,431.819638,14.428334,e7fbe08bb2fe48149ffd829a4a4ddd00.png
4,198.904976,201.996764,290.628019,197.359083,250.434783,312.785821,219.516908,320.515290,275.169082,313.816425,...,357.616739,148.405797,288.566812,349.887271,265.893720,110.789034,135.523334,385.442826,90.177126,b9db262dc66f4ba0b529559f64e0d7d6.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23113,257.500000,175.176152,318.333333,183.848238,271.666667,232.411924,252.500000,223.306233,298.750000,231.978320,...,248.455285,211.666667,155.663957,379.583333,191.219512,215.000000,69.810298,387.916667,102.330623,dbc4fb0f87ea4ff98a8c32ee2a022591.png
23114,228.792271,136.038647,346.280193,143.768116,246.666667,245.420054,298.333333,273.604336,218.333333,278.373984,...,300.921409,177.500000,120.542005,504.583333,202.493225,225.833333,0.867209,470.833333,0.867209,632e2e78d32d447fa584a53bca33fdb1.png
23115,256.103044,285.990338,339.581305,308.663430,271.666667,368.563686,246.250000,367.262873,309.166667,386.341463,...,399.349593,172.916667,265.799458,439.583333,342.113821,230.000000,109.701897,462.500000,170.840108,708029e4c71946fb8004602423884175.png
23116,251.666667,329.105691,363.333333,300.487805,320.416667,350.352304,270.416667,395.013550,370.833333,365.528455,...,380.160000,216.666667,415.392954,427.083333,339.945799,140.833333,291.382114,404.166667,189.051491,77f8e7e784a84965a81f5257299f96e8.png


In [25]:
user_df[user_df.columns[:-1]] = user_df[user_df.columns[:-1]].apply(lambda x: x/2)

cols = list(resized_df.columns)
cols = cols[1:] + [cols[0]]
resized_df = resized_df[cols]

resized_df.columns = user_df.columns

In [26]:
resized_and_user_df = user_df.append(resized_df)

In [27]:
resized_and_user_df

,leftEyePoint_x,leftEyePoint_y,rightEyePoint_x,rightEyePoint_y,centreMouthPoint_x,centreMouthPoint_y,leftMouthPoint_x,leftMouthPoint_y,rightMouthPoint_x,rightMouthPoint_y,...,chinPoint_y,leftHeadPoint_x,leftHeadPoint_y,rightHeadPoint_x,rightHeadPoint_y,leftEarPoint_x,leftEarPoint_y,rightEarPoint_x,rightEarPoint_y,filename
0,88.115942,159.613527,208.953297,162.834131,176.489529,244.251208,147.117548,258.937198,200.708527,251.465374,...,310.466981,62.093394,256.618357,235.748792,242.190012,53.075677,137.842186,216.425121,124.702089,6310b3dbaa3549ea90badf31c7bee933.png
1,84.508853,103.059577,186.022536,127.278575,112.334940,240.386473,92.753623,243.735906,131.143309,249.661836,...,266.924312,48.180350,152.270531,204.830918,177.777778,77.294686,59.774553,219.259251,96.360701,d65b1920c7ea4c61bd7ab8709b7e6d1c.png
2,106.280193,97.004831,158.840580,115.555556,111.304348,150.338164,96.876003,155.104662,123.671498,162.061184,...,170.563599,86.183575,105.507246,177.777778,142.995169,112.334940,24.218998,216.167464,90.692428,943a14d469664950a9d4168eee6e2291.png
3,133.977452,143.768116,204.315616,136.296292,182.157802,240.901763,164.380024,242.190012,197.874396,239.613527,...,269.500797,97.133648,170.048309,234.460544,171.851848,82.190012,19.581317,215.909819,7.214167,e7fbe08bb2fe48149ffd829a4a4ddd00.png
4,99.452488,100.998382,145.314010,98.679541,125.217391,156.392911,109.758454,160.257645,137.584541,156.908213,...,178.808370,74.202899,144.283406,174.943635,132.946860,55.394517,67.761667,192.721413,45.088563,b9db262dc66f4ba0b529559f64e0d7d6.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5984,82.836876,137.600006,105.531914,110.080002,178.156036,81.279999,256.453888,104.960007,276.879425,132.479996,...,113.919998,216.737595,117.119995,144.113480,144.639999,174.751770,138.879990,201.985809,144.639999,yorkshire_terrier_93.jpg
5985,108.160004,81.194031,135.679993,37.253731,193.919998,20.059702,227.199997,69.731346,245.119995,124.179108,...,77.373131,200.319992,94.567169,153.599991,139.462692,175.359985,138.507462,189.440002,151.880600,yorkshire_terrier_96.jpg
5986,131.820892,138.879990,139.462692,77.440002,199.641800,101.759995,278.925385,88.959999,250.268646,136.320007,...,139.519989,219.701508,140.160004,178.626862,172.160004,196.776123,173.440002,207.283569,172.160004,yorkshire_terrier_97.jpg
5987,129.919998,204.799988,147.199997,149.333328,170.240005,181.760010,202.240005,159.573334,204.160004,211.626663,...,215.039993,182.399994,217.600006,155.520004,254.293335,168.959991,251.733337,176.639999,253.440002,yorkshire_terrier_98.jpg


In [28]:
resized_and_user_df.to_csv('../data/resized_and_user_labels.csv', index=False)